In [1]:
# TODO: Remove this
# https://www.kaggle.com/sudalairajkumar/transfer-learning-on-stack-exchange-tags/frequent-words-model-v3/code

In [2]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
stops = set(stopwords.words("english"))

In [3]:
len(stops)

153

In [4]:
DATA_FOLDER = 'data/'
TRAIN_FILES = ['biology.csv', 'cooking.csv', 'crypto.csv', 'diy.csv', 'robotics.csv', 'travel.csv']
TEST_FILE = 'test.csv'

In [5]:
DOMAIN_COUNT = len(TRAIN_FILES)

In [6]:
df_trains = [pd.read_csv(DATA_FOLDER + filename) for filename in TRAIN_FILES]

In [7]:
for df in df_trains:
    print(df.shape)

(13196, 4)
(15404, 4)
(10432, 4)
(25918, 4)
(2771, 4)
(19279, 4)


In [8]:
sum([df.shape[0] for df in df_trains])

87000

In [9]:
df_trains[0].head()

,id,title,content,tags
0,1,What is the criticality of the ribosome bindin...,"<p>In prokaryotic translation, how critical fo...",ribosome binding-sites translation synthetic-b...
1,2,How is RNAse contamination in RNA based experi...,<p>Does anyone have any suggestions to prevent...,rna biochemistry
2,3,Are lymphocyte sizes clustered in two groups?,<p>Tortora writes in <em>Principles of Anatomy...,immunology cell-biology hematology
3,4,How long does antibiotic-dosed LB maintain goo...,<p>Various people in our lab will prepare a li...,cell-culture
4,5,Is exon order always preserved in splicing?,<p>Are there any cases in which the splicing m...,splicing mrna spliceosome introns exons


In [11]:
train_tags = df_trains[0]['tags'].tolist()
train_tags

['ribosome binding-sites translation synthetic-biology',
 'rna biochemistry',
 'immunology cell-biology hematology',
 'cell-culture',
 'splicing mrna spliceosome introns exons',
 'dna biochemistry molecular-biology',
 'neuroscience synapses',
 'plasmids',
 'molecular-genetics gene-expression experimental-design',
 'evolution mitochondria chloroplasts',
 'high-throughput cell-based',
 'molecular-biology synthetic-biology',
 'bioinformatics homework',
 'neuroscience immunology',
 'splicing histone',
 'genomics gene-annotation exons',
 'microbiology virology influenza',
 'epigenetics',
 'molecular-biology dna-isolation',
 'cell-membrane adaptation cell-biology',
 'transcription chromatin',
 'biochemistry neuroscience',
 'molecular-biology transcription rna-interference',
 'neuroscience endocrinology human-biology',
 'bioinformatics phylogenetics',
 'splicing introns genetics',
 'biochemistry rna',
 'microbiology astrobiology',
 'dna primer genetics',
 'microbiology history bacteriology',


In [8]:
# # how many unique tags

# domain_unique_tags = [set() for i in range(DOMAIN_COUNT)]
# unique_tags = set()

# for i in range(DOMAIN_COUNT):
#     df = df_trains[i]
#     posts_tags = df['tags'].tolist()
#     for tags in posts_tags:
#         for tag in tags.split(' '):
#             domain_unique_tags[i].add(tag)
#             unique_tags.add(tag)

# print('domain_unique_tag counts:')
# s = 0
# for tag_set in domain_unique_tags:
#     print(len(tag_set))

# print('sum of domain_unique_tag counts:')
# print(sum([len(tag_set) for tag_set in domain_unique_tags]))
    
# print('all tag count:')
# print(len(unique_tags))


# Cleanse Data

In [130]:
stop_words = {'a', "a's", 'able', 'about', 'above', 'according', 'accordingly',
              'across', 'actually', 'after', 'afterwards', 'again', 'against',
              "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along',
              'already', 'also', 'although', 'always', 'am', 'among', 'amongst',
              'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone',
              'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear',
              'appreciate', 'appropriate', 'are', "aren't", 'around', 'as',
              'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away',
              'awfully', 'b', 'be', 'became', 'because', 'become', 'becomes',
              'becoming', 'been', 'before', 'beforehand', 'behind', 'being',
              'believe', 'below', 'beside', 'besides', 'best', 'better',
              'between', 'beyond', 'both', 'brief', 'but', 'by', 'c', "c'mon",
              "c's", 'came', 'can', "can't", 'cannot', 'cant', 'cause',
              'causes', 'certain', 'certainly', 'changes', 'clearly', 'co',
              'com', 'come', 'comes', 'concerning', 'consequently', 'consider',
              'considering', 'contain', 'containing', 'contains',
              'corresponding', 'could', "couldn't", 'course', 'currently', 'd',
              'definitely', 'described', 'despite', 'did', "didn't",
              'different', 'do', 'does', "doesn't", 'doing', "don't", 'done',
              'down', 'downwards', 'during', 'e', 'each', 'edu', 'eg', 'eight',
              'either', 'else', 'elsewhere', 'enough', 'entirely', 'especially',
              'et', 'etc', 'even', 'ever', 'every', 'everybody', 'everyone',
              'everything', 'everywhere', 'ex', 'exactly', 'example', 'except',
              'f', 'far', 'few', 'fifth', 'first', 'five', 'followed',
              'following', 'follows', 'for', 'former', 'formerly', 'forth',
              'four', 'from', 'further', 'furthermore', 'g', 'get', 'gets',
              'getting', 'given', 'gives', 'go', 'goes', 'going', 'gone', 'got',
              'gotten', 'greetings', 'h', 'had', "hadn't", 'happens', 'hardly',
              'has', "hasn't", 'have', "haven't", 'having', 'he', "he's",
              'hello', 'help', 'hence', 'her', 'here', "here's", 'hereafter',
              'hereby', 'herein', 'hereupon', 'hers', 'herself', 'hi', 'him',
              'himself', 'his', 'hither', 'hopefully', 'how', 'howbeit',
              'however', 'i', "i'd", "i'll", "i'm", "i've", 'ie', 'if',
              'ignored', 'immediate', 'in', 'inasmuch', 'inc', 'indeed',
              'indicate', 'indicated', 'indicates', 'inner', 'insofar',
              'instead', 'into', 'inward', 'is', "isn't", 'it', "it'd", "it'll",
              "it's", 'its', 'itself', 'j', 'just', 'k', 'keep', 'keeps',
              'kept', 'know', 'knows', 'known', 'l', 'last', 'lately', 'later',
              'latter', 'latterly', 'least', 'less', 'lest', 'let', "let's",
              'like', 'liked', 'likely', 'little', 'look', 'looking', 'looks',
              'ltd', 'm', 'mainly', 'many', 'may', 'maybe', 'me', 'mean',
              'meanwhile', 'merely', 'might', 'more', 'moreover', 'most',
              'mostly', 'much', 'must', 'my', 'myself', 'n', 'name', 'namely',
              'nd', 'near', 'nearly', 'necessary', 'need', 'needs', 'neither',
              'never', 'nevertheless', 'new', 'next', 'nine', 'no', 'nobody',
              'non', 'none', 'noone', 'nor', 'normally', 'not', 'nothing',
              'novel', 'now', 'nowhere', 'o', 'obviously', 'of', 'off', 'often',
              'oh', 'ok', 'okay', 'old', 'on', 'once', 'one', 'ones', 'only',
              'onto', 'or', 'other', 'others', 'otherwise', 'ought', 'our',
              'ours', 'ourselves', 'out', 'outside', 'over', 'overall', 'own',
              'p', 'particular', 'particularly', 'per', 'perhaps', 'placed',
              'please', 'plus', 'possible', 'presumably', 'probably',
              'provides', 'q', 'que', 'quite', 'qv', 'r', 'rather', 'rd', 're',
              'really', 'reasonably', 'regarding', 'regardless', 'regards',
              'relatively', 'respectively', 'right', 's', 'said', 'same', 'saw',
              'say', 'saying', 'says', 'second', 'secondly', 'see', 'seeing',
              'seem', 'seemed', 'seeming', 'seems', 'seen', 'self', 'selves',
              'sensible', 'sent', 'serious', 'seriously', 'seven', 'several',
              'shall', 'she', 'should', "shouldn't", 'since', 'six', 'so',
              'some', 'somebody', 'somehow', 'someone', 'something', 'sometime',
              'sometimes', 'somewhat', 'somewhere', 'soon', 'sorry',
              'specified', 'specify', 'specifying', 'still', 'sub', 'such',
              'sup', 'sure', 't', "t's", 'take', 'taken', 'tell', 'tends', 'th',
              'than', 'thank', 'thanks', 'thanx', 'that', "that's", 'thats',
              'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence',
              'there', "there's", 'thereafter', 'thereby', 'therefore',
              'therein', 'theres', 'thereupon', 'these', 'they', "they'd",
              "they'll", "they're", "they've", 'think', 'third', 'this',
              'thorough', 'thoroughly', 'those', 'though', 'three', 'through',
              'throughout', 'thru', 'thus', 'to', 'together', 'too', 'took',
              'toward', 'towards', 'tried', 'tries', 'truly', 'try', 'trying',
              'twice', 'two', 'u', 'un', 'under', 'unfortunately', 'unless',
              'unlikely', 'until', 'unto', 'up', 'upon', 'us', 'use', 'used',
              'useful', 'uses', 'using', 'usually', 'uucp', 'v', 'value',
              'various', 'very', 'via', 'viz', 'vs', 'w', 'want', 'wants',
              'was', "wasn't", 'way', 'we', "we'd", "we'll", "we're", "we've",
              'welcome', 'well', 'went', 'were', "weren't", 'what', "what's",
              'whatever', 'when', 'whence', 'whenever', 'where', "where's",
              'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon',
              'wherever', 'whether', 'which', 'while', 'whither', 'who',
              "who's", 'whoever', 'whole', 'whom', 'whose', 'why', 'will',
              'willing', 'wish', 'with', 'within', 'without', "won't", 'wonder',
              'would', 'would', "wouldn't", 'x', 'y', 'yes', 'yet', 'you',
              "you'd", "you'll", "you're", "you've", 'your', 'yours',
              'yourself', 'yourselves', 'z', 'zero', ''}

len(stop_words)

571

In [105]:
import re
from bs4 import BeautifulSoup
# import nltk
# tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

# remove [n]
# remove pure numbers, don't remove letter+number words, like CO2

# remove stopwords
# consider µL?
# remove formulas

def cleanse_html(content):
    content = BeautifulSoup(content, "lxml").get_text()

    # remove urls
    content = re.sub(r"\S+:/\S+","", content)  # "xxx:/xxx"
    content = re.sub(r"\S+\\\S+","", content)  # "xxx\xxx"
    
    return content
    

def cleanse_lower_split(content):
    # remove html tags
    content = BeautifulSoup(content, "lxml").get_text()
    
    # remove urls
    content = re.sub(r"\S+:/\S+","", content)  # "xxx:/xxx"
    content = re.sub(r"\S+\\\S+","", content)  # "xxx\xxx"
    
    # replace punctuations and numbers with whitespaces
    content = re.sub(r"[^a-zA-Z0-9\-]"," ", content)
        
    # convert to lowercase
    content = content.lower()
    
    words = content.split()
    
    # remove useless words
    # stopwords, pure numbers, short words
    words = [word for word in words if not (word in stop_words or len(word) < 3 or not re.search('[a-z]', word))]
    
    return words

numbers = set('0123456789-')

def is_useful_word(word):
    return (word not in stop_words) and (len(word) > 2) and (re.search('[a-z]', word)) and (word[0] not in numbers)
    
# only reserve words.
def cleanse(content):
    # remove html tags
    content = BeautifulSoup(content, "lxml").get_text()
    
    # remove urls
    content = re.sub(r"\S+:/\S+","", content)  # "xxx:/xxx"
    content = re.sub(r"\S+\\\S+","", content)  # "xxx\xxx"
    
    # replace punctuations and numbers with whitespaces
    content = re.sub(r"[^a-zA-Z0-9\-]"," ", content)
        
    # convert to lowercase
    content = content.lower()
    
    words = content.split()
    
    # remove useless words
    # stopwords, pure numbers, short words
    words = [word for word in words if is_useful_word(word)]
    
    # concat words to a string
    content = ' '.join(words)
    
    return content

In [106]:
df_test = pd.read_csv(DATA_FOLDER + TEST_FILE)
print(df_test.shape)
df_test.head()

(81926, 3)


,id,title,content
0,1,What is spin as it relates to subatomic partic...,<p>I often hear about subatomic particles havi...
1,2,What is your simplest explanation of the strin...,<p>How would you explain string theory to non ...
2,3,"Lie theory, Representations and particle physics",<p>This is a question that has been posted at ...
3,7,Will Determinism be ever possible?,<p>What are the main problems that we need to ...
4,9,Hamilton's Principle,<p>Hamilton's principle states that a dynamic ...


In [107]:
def cleanse_contents_of_df(df):
    contents = df['content'].tolist()
    cleansed_contents = [cleanse(content) for content in contents]
    return cleansed_contents

In [ ]:
def predict_tags_from_contents(df):
    contents = df['content'].tolist()
    cleansed_contents = [cleanse(content) for content in contents]

In [108]:
cleansed_test_contents = cleanse_contents_of_df(df_test)
cleansed_test_contents[:10]

['hear subatomic particles property called spin doesn relate spinning axis particles spin spin actual spinning motion',
 'explain string theory physicists specially interested plausible needed successfully prove',
 'question posted forums thought conceptual answer physicists care representations lie groups representation means sort group acting vector space vector space lie group acting things invariant group action dumb question thought good start clarify specifically thinking symmetry groups people relation standard model care group group acting acting',
 'main problems solve prove laplace determinism correct overcome uncertainty principle',
 'hamilton principle states dynamic system path action integral stationary maximum minimum action integral stationary basis hamilton state principle',
 'term sound life clue sound created sound produced measured',
 'big controversy groups physicists support string theory oppose arguments group disprove correctness string theory question defined e

# Predict Tags

In [132]:

vectorizer = TfidfVectorizer(use_idf=False, \
                             stop_words = "english", \
                             tokenizer = str.split,    \
                             preprocessor = None, \
                             max_features = 10000)

In [133]:
test_features = vectorizer.fit_transform(cleansed_test_contents)
test_features.shape
print(test_features[0])

  (0, 5708)	0.196116135138
  (0, 102)	0.196116135138
  (0, 624)	0.196116135138
  (0, 8365)	0.392232270276
  (0, 7386)	0.196116135138
  (0, 2615)	0.196116135138
  (0, 8353)	0.588348405415
  (0, 1081)	0.196116135138
  (0, 6952)	0.196116135138
  (0, 6371)	0.392232270276
  (0, 8611)	0.196116135138
  (0, 4056)	0.196116135138


In [134]:
feature_words = vectorizer.get_feature_names()

In [135]:
_, word_idxs = test_features[0].nonzero()
print(word_idxs)

[5708  102  624 8365 7386 2615 8353 1081 6952 6371 8611 4056]


In [140]:
for word_idx in word_idxs:
    print(feature_words[word_idx])

motion
actual
axis
spinning
relate
doesn
spin
called
property
particles
subatomic
hear


In [147]:
predicted_tags = []

for feature in test_features:
    values = feature.data
    _, word_idxs = feature.nonzero()
    
    index_sorted = np.argsort(values)[::-1]
    keywords = [feature_words[wid] for wid in word_idxs[index_sorted[:3]]]
    predicted_tags.append(' '.join(keywords))

In [148]:
predicted_tags[:20]

['spin particles spinning',
 'explain string theory',
 'group acting vector',
 'prove main problems',
 'action principle hamilton',
 'sound term life',
 'string theory disprove',
 'sky change color',
 'energy higher particles',
 'monte carlo method',
 'bike turn left',
 'formula projectile wondering',
 'particle wave function',
 'miles average trip',
 'theory special person',
 'direction common water',
 'energy magnets poles',
 'theories world mathematical',
 'field theories physicists',
 'properties body contact']

# procedure

* cleanse title and contents
* apply tf idf

## try to guess if a word is a keyword

* train the guess function with training data
* input: tf, idf of title, content

## try to guess the tags of a post



In [ ]:
# remove [n]
# remove pure numbers, don't remove letter+number words, like CO2
# remove urls

In [ ]:
# tf-idf over all contents
# same-category contents concat as one doc

In [ ]:
# for each tag, find words that mostly exist in title and contents of that tag
# use entropy to do the above thing
# if a test content contains words only exist in , then the content is very likely about that domain

In [ ]:
# as classification problem
# features: bow (normalized)
